## Package Imports

In [104]:
# general imports
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from convokit import Corpus, download
from tqdm import tqdm
import nltk

# set up nltk tokenizers
from nltk.tokenize import word_tokenize, sent_tokenize, TweetTokenizer
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')

# set up nltk lemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk import pos_tag
from nltk.corpus import wordnet

# imports specific to lexical measures
import re
from wordfreq import zipf_frequency
from lexical_diversity import lex_div as ld
from collections import Counter


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/nickvick/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nickvick/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nickvick/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/nickvick/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Format Data

In [105]:
corpus = Corpus(filename=download("subreddit-Cornell"))

Dataset already exists at /Users/nickvick/.convokit/saved-corpora/subreddit-Cornell


In [106]:
def corpus_to_df(corpus):

    data = []
    for utt in corpus.iter_utterances():
        # only consider utterances with timestamps and text
        if hasattr(utt, "timestamp") and utt.text:
            # convert timestamp from seconds since 1/1/1970 to datetime
            t = datetime.fromtimestamp(int(utt.timestamp))

            data.append({
                "utterance_id": utt.id,
                "speaker_id": utt.speaker.id,
                "text": utt.text,
                "timestamp": t
            })

    df = pd.DataFrame(data)
    return df

## DF Level Cleaning

In [107]:
BOT_TEXT_PATTERNS = [
    r"\bi am a bot\b",
    r"\bthis (comment|post) was (posted|left by) a bot",
    r"\bthis reply was generated automatically",
    r"[\^*]*beep(?:\s+beep)?[\^*]*\s+[\^*]*boop(?:\s+boop)?[\^*]*"
]

BOT_TEXT_RE = re.compile("|".join(BOT_TEXT_PATTERNS), flags=re.IGNORECASE)
URL_RE = re.compile(r"http\S+|www\.\S+")
HAS_LETTER_RE = re.compile(r"[A-Za-z]")

## Cleaning

In [108]:
def clean_text(text):

    # remove urls
    text = URL_RE.sub("", text)

    return text

In [109]:
def tokenize(text):
    '''Helper function to tokenize social media text. Note that the TweetTokenizer 
    preserves mentions, contractions, and other social media-specific structures'''

    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)

    return tokens

In [119]:
def get_wordnet_pos(treebank_tag):
    
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [112]:
def lemmatize(tokens):
    '''Helper function to lemmatize tokens.'''

    tagged = pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(tok, get_wordnet_pos(tag)) for tok, tag in tagged]

    return lemmatized_tokens

In [113]:
def clean_tokens_lexical(text):
    '''Helper function that tokenizes text, cleans tokens by removing punctuation and numbers
    for purely lexical analysis, and returns the cleaned, lemmatized tokens.'''
    
    # clean text
    text = clean_text(text)

    # tokenize text
    tokens = tokenize(text)

    # clean tokens
    cleaned = []
    for tok in tokens:
        # skip over punctuation
        if re.match(r'^\W+$', tok):
            continue

        # only keep alphabetic tokens
        if tok.isalpha():
            cleaned.append(tok.lower())

    # lemmatize clean tokens
    lemmatized = lemmatize(cleaned)

    return lemmatized

## Preprocess Function

In [114]:
def preprocess_df(df):
    # remove deleted/removed utterances
    df = df[~df["text"].str.lower().isin({"[deleted]", "[removed]"})]

    # remove bot authored utterances
    df = df[~df["text"].str.contains(BOT_TEXT_RE)]

    # remove utterances without a letter
    df = df[df["text"].str.contains(HAS_LETTER_RE)]

    # tokenize
    df["tokens"] = df["text"].apply(tokenize)

    # lemmatize
    df["lemmas"] = df["tokens"].apply(lemmatize)

    # final
    df["final"] = df["text"].apply(clean_tokens_lexical)

    return df

## Testing

In [120]:
import random

In [121]:
utterance = list(corpus.iter_utterances())
sample_utts = random.sample(utterance, 100)

In [122]:
pd.set_option('display.max_rows', 100)

In [123]:
df = corpus_to_df(corpus)
df = preprocess_df(df)

df

/var/folders/dj/6y_wgvgs5m59w08whyk9sjp80000gn/T/ipykernel_27295/2076697257.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df["text"].str.contains(BOT_TEXT_RE)]


,utterance_id,speaker_id,text,timestamp,tokens,lemmas,final
0,nyx4d,reddmau5,I was just reading about the Princeton Mic-Check and it's getting [national press](http://www.bloomberg.com/news/2011-12-29/princeton-brews-trouble-for-us-1-percenters-commentary-by-michael-lewis.html).\n\nI want to get a sense of what people felt like around campus. Anything interesting happen? Anything interesting coming up?,2012-01-01 16:18:18,"[I, was, just, reading, about, the, Princeton, Mic-Check, and, it's, getting, [, national, press, ], (, http://www.bloomberg.com/news/2011-12-29/princeton-brews-trouble-for-us-1-percenters-commentary-by-michael-lewis.html, ), ., I, want, to, get, a, sense, of, what, people, felt, like, around, campus, ., Anything, interesting, happen, ?, Anything, interesting, coming, up, ?]","[I, be, just, read, about, the, Princeton, Mic-Check, and, it's, get, [, national, press, ], (, http://www.bloomberg.com/news/2011-12-29/princeton-brews-trouble-for-us-1-percenters-commentary-by-michael-lewis.html, ), ., I, want, to, get, a, sense, of, what, people, felt, like, around, campus, ., Anything, interesting, happen, ?, Anything, interest, come, up, ?]","[i, be, just, read, about, the, princeton, and, get, national, press, i, want, to, get, a, sense, of, what, people, felt, like, around, campus, anything, interesting, happen, anything, interest, come, up]"
1,o0145,shtylman,"I have added support for Cornell to courseoff.com (https://cornell.courseoff.com). Courseoff is a free web app to help you plan your semester schedules. It is very popular with students at some of the other schools I support.\n\nNo signup is required to use it so feel free to try it out! You can create an account which allows multiple schedules, saving schedules, and sharing schedules.\n\nLet me know what you guys think! Any feedback is always appreciated. If you like it, tell your friends :) If you find a problem, let me know as well.",2012-01-02 13:57:15,"[I, have, added, support, for, Cornell, to, courseoff.com, (, https://cornell.courseoff.com, ), ., Courseoff, is, a, free, web, app, to, help, you, plan, your, semester, schedules, ., It, is, very, popular, with, students, at, some, of, the, other, schools, I, support, ., No, signup, is, required, to, use, it, so, feel, free, to, try, it, out, !, You, can, create, an, account, which, allows, multiple, schedules, ,, saving, schedules, ,, and, sharing, schedules, ., Let, me, know, what, you, guys, think, !, Any, feedback, is, always, appreciated, ., If, you, like, it, ,, tell, your, friends, :), If, you, find, a, ...]","[I, have, add, support, for, Cornell, to, courseoff.com, (, https://cornell.courseoff.com, ), ., Courseoff, be, a, free, web, app, to, help, you, plan, your, semester, schedule, ., It, be, very, popular, with, student, at, some, of, the, other, school, I, support, ., No, signup, be, require, to, use, it, so, feel, free, to, try, it, out, !, You, can, create, an, account, which, allow, multiple, schedule, ,, save, schedule, ,, and, share, schedule, ., Let, me, know, what, you, guy, think, !, Any, feedback, be, always, appreciate, ., If, you, like, it, ,, tell, your, friend, :), If, you, find, a, ...]","[i, have, add, support, for, cornell, to, courseoff, be, a, free, web, app, to, help, you, plan, your, semester, schedule, it, be, very, popular, with, student, at, some, of, the, other, school, i, support, no, signup, be, require, to, use, it, so, feel, free, to, try, it, out, you, can, create, an, account, which, allow, multiple, schedule, save, schedule, and, share, schedule, let, me, know, what, you, guy, think, any, feedback, be, always, appreciate, if, you, like, it, tell, your, friend, if, you, find, a, problem, let, me, know, as, well]"
2,o1gca,moon_river,"i don't have a facebook, so we'd need a volunteer.. just someone to let cornell on facebook know that we have a presence on reddit.. perhaps a small explanation of what reddit is? now that we are almost beautiful and such.. we need 

In [124]:
df_text = df[["text", "tokens", "lemmas", "final"]]
df_text

,text,tokens,lemmas,final
0,I was just reading about the Princeton Mic-Check and it's getting [national press](http://www.bloomberg.com/news/2011-12-29/princeton-brews-trouble-for-us-1-percenters-commentary-by-michael-lewis.html).\n\nI want to get a sense of what people felt like around campus. Anything interesting happen? Anything interesting coming up?,"[I, was, just, reading, about, the, Princeton, Mic-Check, and, it's, getting, [, national, press, ], (, http://www.bloomberg.com/news/2011-12-29/princeton-brews-trouble-for-us-1-percenters-commentary-by-michael-lewis.html, ), ., I, want, to, get, a, sense, of, what, people, felt, like, around, campus, ., Anything, interesting, happen, ?, Anything, interesting, coming, up, ?]","[I, be, just, read, about, the, Princeton, Mic-Check, and, it's, get, [, national, press, ], (, http://www.bloomberg.com/news/2011-12-29/princeton-brews-trouble-for-us-1-percenters-commentary-by-michael-lewis.html, ), ., I, want, to, get, a, sense, of, what, people, felt, like, around, campus, ., Anything, interesting, happen, ?, Anything, interest, come, up, ?]","[i, be, just, read, about, the, princeton, and, get, national, press, i, want, to, get, a, sense, of, what, people, felt, like, around, campus, anything, interesting, happen, anything, interest, come, up]"
1,"I have added support for Cornell to courseoff.com (https://cornell.courseoff.com). Courseoff is a free web app to help you plan your semester schedules. It is very popular with students at some of the other schools I support.\n\nNo signup is required to use it so feel free to try it out! You can create an account which allows multiple schedules, saving schedules, and sharing schedules.\n\nLet me know what you guys think! Any feedback is always appreciated. If you like it, tell your friends :) If you find a problem, let me know as well.","[I, have, added, support, for, Cornell, to, courseoff.com, (, https://cornell.courseoff.com, ), ., Courseoff, is, a, free, web, app, to, help, you, plan, your, semester, schedules, ., It, is, very, popular, with, students, at, some, of, the, other, schools, I, support, ., No, signup, is, required, to, use, it, so, feel, free, to, try, it, out, !, You, can, create, an, account, which, allows, multiple, schedules, ,, saving, schedules, ,, and, sharing, schedules, ., Let, me, know, what, you, guys, think, !, Any, feedback, is, always, appreciated, ., If, you, like, it, ,, tell, your, friends, :), If, you, find, a, ...]","[I, have, add, support, for, Cornell, to, courseoff.com, (, https://cornell.courseoff.com, ), ., Courseoff, be, a, free, web, app, to, help, you, plan, your, semester, schedule, ., It, be, very, popular, with, student, at, some, of, the, other, school, I, support, ., No, signup, be, require, to, use, it, so, feel, free, to, try, it, out, !, You, can, create, an, account, which, allow, multiple, schedule, ,, save, schedule, ,, and, share, schedule, ., Let, me, know, what, you, guy, think, !, Any, feedback, be, always, appreciate, ., If, you, like, it, ,, tell, your, friend, :), If, you, find, a, ...]","[i, have, add, support, for, cornell, to, courseoff, be, a, free, web, app, to, help, you, plan, your, semester, schedule, it, be, very, popular, with, student, at, some, of, the, other, school, i, support, no, signup, be, require, to, use, it, so, feel, free, to, try, it, out, you, can, create, an, account, which, allow, multiple, schedule, save, schedule, and, share, schedule, let, me, know, what, you, guy, think, any, feedback, be, always, appreciate, if, you, like, it, tell, your, friend, if, you, find, a, problem, let, me, know, as, well]"
2,"i don't have a facebook, so we'd need a volunteer.. just someone to let cornell on facebook know that we have a presence on reddit.. perhaps a small explanation of what reddit is? now that we are almost beautiful and such.. we need more redditors!","[i, don't, have, a, facebook, ,, so, we'd, need, a, volunteer, .., just, someone, to, let, cornell, on,

In [125]:
pd.set_option('display.max_colwidth', None)
df_text.sample(50)

,text,tokens,lemmas,final
25789,"Don't know if a younger premed can chime in, but at least when I took psych 1101, it didn't correspond well with the experimental psych section (I took the old MCAT). I think I heard that you will need to do some extra work on your own for MCAT psych.","[Don't, know, if, a, younger, premed, can, chime, in, ,, but, at, least, when, I, took, psych, 1101, ,, it, didn't, correspond, well, with, the, experimental, psych, section, (, I, took, the, old, MCAT, ), ., I, think, I, heard, that, you, will, need, to, do, some, extra, work, on, your, own, for, MCAT, psych, .]","[Don't, know, if, a, young, premed, can, chime, in, ,, but, at, least, when, I, take, psych, 1101, ,, it, didn't, correspond, well, with, the, experimental, psych, section, (, I, take, the, old, MCAT, ), ., I, think, I, hear, that, you, will, need, to, do, some, extra, work, on, your, own, for, MCAT, psych, .]","[know, if, a, young, premed, can, chime, in, but, at, least, when, i, take, psych, it, correspond, well, with, the, experimental, psych, section, i, take, the, old, mcat, i, think, i, hear, that, you, will, need, to, do, some, extra, work, on, your, own, for, mcat, psych]"
38749,Professor White just sent out an email offering people consultant positions so I would check your cornell email.,"[Professor, White, just, sent, out, an, email, offering, people, consultant, positions, so, I, would, check, your, cornell, email, .]","[Professor, White, just, send, out, an, email, offering, people, consultant, position, so, I, would, check, your, cornell, email, .]","[professor, white, just, send, out, an, email, offering, people, consultant, position, so, i, would, check, your, cornell, email]"
21156,"It'll vary by class whether or not an older edition is fine, so I'd ask the professor for each class. And I don't know if this is the best place for it, but people buy sell textbooks on the official Facebook groups fairly often, so you could try there.","[It'll, vary, by, class, whether, or, not, an, older, edition, is, fine, ,, so, I'd, ask, the, professor, for, each, class, ., And, I, don't, know, if, this, is, the, best, place, for, it, ,, but, people, buy, sell, textbooks, on, the, official, Facebook, groups, fairly, often, ,, so, you, could, try, there, .]","[It'll, vary, by, class, whether, or, not, an, old, edition, be, fine, ,, so, I'd, ask, the, professor, for, each, class, ., And, I, don't, know, if, this, be, the, best, place, for, it, ,, but, people, buy, sell, textbook, on, the, official, Facebook, group, fairly, often, ,, so, you, could, try, there, .]","[vary, by, class, whether, or, not, an, old, edition, be, fine, so, ask, the, professor, for, each, class, and, i, know, if, this, be, the, best, place, for, it, but, people, buy, sell, textbook, on, the, official, facebook, group, fairly, often, so, you, could, try, there]"
36291,"Current ChemE rising junior that loves research here, I'd say to start as soon as you can! Research is such a valuable thing to spend your time on at Cornell, not just for the benefits mentioned above but for the satisfaction of contributing to something greater than yourself! Find a professor in the ChemE department (or any department really) whose research interests you and email them saying:\n\nWho you are\nWhy their research interests you\nCan I come by your office to talk about it more\n\nThis is your best bet because if they do have positions open, you can have a face to face way of asking about it which puts your face in their head making your chances better. If they don't have any positions open, you'll still have made that connection by talking in person and they can point you to another faculty member that might have an open spot or might even better suit your interests! Good luck finding a lab, shoot me a message if you want more personalized help!","[Current, ChemE, rising, junior, that, loves, research, here, ,, I'd, say, to, start, as, soon, as, you, can, !, Research, is, such, a, val